In [270]:
import numpy as np
from matplotlib import pyplot as plt

Carro1 = np.empty((10,3), dtype=float)
Carro2 = np.empty((10,3), dtype=float)

#Ford Fusion
Carro1[0] = [17058, 99900, -1]
Carro1[1] = [16000, 103900, -1]
Carro1[2] = [44667, 104990, -1]
Carro1[3] = [47000, 107900, -1]
Carro1[4] = [19726, 124990, -1]
Carro1[5] = [19826, 116900, -1]
Carro1[6] = [17700, 118900, -1]
Carro1[7] = [28204, 119890, -1]
Carro1[8] = [17000, 120950, -1]
Carro1[9] = [100, 110000, -1]

#Fiat Uno
Carro2[0] = [29501, 34000, 1]
Carro2[1] = [42067, 33440, 1]
Carro2[2] = [34109, 31100, 1]
Carro2[3] = [34713, 31300, -1]
Carro2[4] = [27912, 32499, 1]
Carro2[5] = [28272, 31990, 1]
Carro2[6] = [26733, 32990, 1]
Carro2[7] = [24990, 33660, 1]
Carro2[8] = [16000, 33800, 1]
Carro2[9] = [15000, 35900, 1]

Entrada = np.append(Carro1, Carro2, axis=0)

class Perceptron:
    #Função de inicialização da classe
    def __init__(self, n_entradas, func_act):
        self.pesos = np.empty((n_entradas, 1), dtype=float)
        self.func_act = func_act
            
    def normalizar(self, entradas, utilizar_parametros_anteriores):
        if utilizar_parametros_anteriores == False:
            #Encontrar média e desvio padrão a partir das entradas fornecidas.
            #Executado antes do treinamento da rede
            self.media = np.empty(len(entradas[0,:]), dtype=float)
            self.desvpad = np.empty(len(entradas[0,:]), dtype=float)
            self.entr_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
        
            for i in range(0, len(entradas[0,:])):
                self.media[i] = np.mean(entradas[:,i])
                self.desvpad[i] = np.sqrt(np.var(entradas[:,i]))            
                self.entr_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]
        else:
            #Normaliza os dados de entrada fornecidos conforme a média e desvio padrão já encontrados anteriormente
            entr_nova_normalizada = np.empty((len(entradas[:,0]), len(entradas[0,:])), dtype=float)
            for i in range(0, len(entradas[0,:])):                
                entr_nova_normalizada[:,i] = (entradas[:,i] - self.media[i] ) / self.desvpad[i]            
            return entr_nova_normalizada
            
    def proc_saida(self, entradas, normalizado):
        if not normalizado:
            #Normalização das entradas utilizando média e desvpad já encontrados
            entr_nova_normalizada = self.normalizar(entradas, True)
        else:
            entr_nova_normalizada = entradas
        
        v = np.dot(entr_nova_normalizada, self.pesos)
        if(self.func_act == 'Bipolar'):
            if(v > 0):
                self.saida = 1
                return 1
            else:
                self.saida = -1
                return -1
    
    def treinar_rede(self, fator_aprendizado, saida_desejada, max_iterações):
        redeTreinada = False
        historicoAcertos = []
        historicoPesos = np.array([len(self.pesos), 1])
        
        for i in range(0, max_iterações):
            if not redeTreinada:
                flagAlteração = False
                acertos = 0
                for j in range(len(self.entr_normalizada[:,0])):
                    #Testa as entradas até encontrar um erro
                    #Caso encontrado, continua testando as outras entradas para obter a quantidade de acertos para
                    #os pesos atuais. Em seguida atualiza os pesos e reinicia os testes.
                    res = saida_desejada[j,0] - self.proc_saida(self.entr_normalizada[j,:], True)
                    if res == 0:
                        acertos += 1
                    else:
                        deltaW = fator_aprendizado * res * self.entr_normalizada[j,:]                    
                        flagAlteração = True  
                        
                        #Termina de checar a quantidade de acertos
                        #antes de atualizar os pesos
                        for k in range(j+1, len(self.entr_normalizada[:,0])):
                            res = saida_desejada[k,0] - self.proc_saida(self.entr_normalizada[k,:], True)
                            if res == 0:
                                acertos += 1
                        
                        #Salva os acertos e pesos
                        historicoAcertos.append(acertos)
                        historicoPesos = np.append(historicoPesos, self.pesos, axis=1)
                        
                        #Atualiza os pesos                               
                        self.pesos += np.transpose([deltaW])
                        break
                
                if flagAlteração == False:
                    redeTreinada = True
        
        if redeTreinada == False:
            #Se não foi possível obter 100% de acerto durante as iterações
            #encontra e define como pesos os que obtiveram mais acertos
            indiceMax = historicoAcertos.index(max(historicoAcertos))                        
            self.pesos = np.transpose([historicoPesos[:, indiceMax+1]])
            print('Nº erros: {}'.format(len(self.entr_normalizada[:,0])-max(historicoAcertos)))
        
# -----------------------------------------------------------------------------------------------------
def plotarGrafico(entradas, s_desejada, pesos, titulo):
    plt.figure(figsize=(4,4))
    for i, el in enumerate(entradas):
        if s_desejada[i,0] == -1:
            plt.plot(el[0], el[1], '.b')
        else:
            plt.plot(el[0], el[1], '.r')

    plt.title(titulo)
    
    print(pesos)
    if pesos.any() != None:
        x1 = np.array([-2.1, 2.1])
        x2 = (-1*x1*pesos[0,0])/pesos[1,0]
        plt.plot(x1, x2, '--g')
        plt.plot([0, pesos[0,0]], [0, pesos[1,0]])
    plt.show()


# -----------------------------------------------------------------------------------------------------
perc = Perceptron(n_entradas=2, func_act='Bipolar')
perc.pesos = np.transpose([[0.5, -0.6]])

sd = np.transpose([Entrada[:,2]])
Entradas = Entrada[:,:2]

perc.normalizar(Entradas, utilizar_parametros_anteriores=False)
perc.treinar_rede(0.3, sd, 20)

#print(perc.normalizar(np.array([[4000, 103000]]), True))
#print(perc.proc_saida(np.array([[4000, 103000]]), False))
#print(perc.pesos)

#plt.plot(perc.entr_normalizada[:,0], perc.entr_normalizada[:,1], '.b')

#x1 = np.array([-2.1, 2.1])
#x2 = (-1*x1*perc.pesos[0,0])/perc.pesos[1,0]

#plt.plot(x1, x2, '--g')

plotarGrafico(perc.entr_normalizada, sd, perc.pesos, 'Titulo')

AxisError: axis 1 is out of bounds for array of dimension 1